<a href="https://colab.research.google.com/github/Cutlets/CSV_PP/blob/main/Cosmetic_Data_Preprocess_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recod 전성분 전처리기

In [1]:
# 모듈 목록
import pandas as pd
import numpy as np
import io
import re
from random import *

In [8]:
# 정규화 함수
def re_data(tgr):
  modtext = tgr

  # 단순 제거
  p = [] # 정규식

  p.append('^\-') # 맨 앞 하이픈 처리
  p.append('(\d|\.)+\%\s*') # x% 처리
  p.append('\™|\*|\®') # 특수문자 제거
  p.append('\-?.*Division\:.*Ingredients\:\s') # 에스티로더 라벨 제거

  for i in range(0, len(p)):
    modtext = re.sub(re.compile(p[i]), "", modtext)

  # 문자 치환
  p2 = [] # 정규식
  p3 = [] # 치환할 내용

  #괄호 먼저 제거
  p2.append('\s*(\(|\]).*?(\)|\])\s*')
  p3.append(' ')

  p2.append('(\s\-)|(\,\-)') # 하이픈 처리
  p3.append(',')

  p2.append('\-\s') # 하이픈처리2
  p3.append('\-')

  p2.append('\sand\s') # and 처리
  p3.append(', ')

  p2.append('\:(\w|\s|\,|\-)*\.') # :로 일반적인 설명 하는 경우
  p3.append(', ')

  p2.append('\/\s*') # /(슬래시) 처리
  p3.append('_')

  p2.append('(?P<fn>\d+)\,(?P<sn>\d+)') # 화합물 숫자 처리 1
  p3.append('\g<fn>-\g<sn>')

  p2.append('(\s|\.)\,') # 특수문자 반복 처리 1
  p3.append(',')

  p2.append('\,\,') # 특수문자 반복 처리 2
  p3.append(',')

  p2.append('\s\s') # 이중공백 처리
  p3.append('s')

  for i in range(0, len(p2)):
    modtext = re.sub(re.compile(p2[i]), p3[i], modtext)

  # NaN 처리
  p4 = [] # 정규식

  p4.append('^Visit.*')
  p4.append('^No Info.*')
  p4.append('\#NAME\?')
  p4.append('^REN.*')

  for i in range(0, len(p4)):
    modtext = re.sub(re.compile(p4[i]), 'NaN', modtext)

  # 공백 제거
  modtext = re.sub(re.compile('\s'), "", modtext)

  return modtext

In [9]:
# 파일은 Github에서 가져옵니다.
# 가져온 데이터는 c_data에 저장
url = 'https://raw.githubusercontent.com/Cutlets/CSV_PP/main/cosmetics_original.csv'
c_data = pd.read_csv(url)

# 확인용
# print(c_data)


In [10]:
# 전처리를 진행할 인덱스를 선택합니다
# 샘플 데이터의 인덱스는 Ingredients 입니다.
# select_index = input('전처리를 진행할 성분을 포함한 인덱스 이름 : ')
select_index = 'Ingredients'
c_len = len(c_data.index)
# 테스트용
k = 0
# c_len = 5

for i in range(k, k + c_len):
  target_data = c_data.loc[:, [select_index]]
  target_text = target_data.iloc[i][0]
  # print('=========')
  # print(target_text)
  # print('---------')
  mod_text = re_data(target_text)
  # print(mod_text)
  # print('=========')
  if mod_text == "NaN":
    c_data.loc[i, select_index] = np.nan
  else:
    c_data.loc[i, select_index] = mod_text

m_cosdata = c_data.dropna()

In [11]:
# 내보내기를 진행할건지 확인받음
isImport = int(input('내보내기를 진행하려면 1을 입력해주세요 : '))
if isImport == 1:
  print('CSV로 내보내는 중...')
  # CSV 형식으로 구분자는 ,로 표현하여 내보냄
  m_cosdata.to_csv('cosmetics_modified.csv', sep=',', encoding = 'UTF-8')
  print('CSV로 내보내기를 진행하였습니다.')
else:
  print('내보내기를 진행하지 않았습니다.')

#내보낸 파일이름은 'cosmetics_modified.csv' 입니다.

내보내기를 진행하려면 1을 입력해주세요 : 1
CSV로 내보내는 중...
CSV로 내보내기를 진행하였습니다.
